<a href="https://colab.research.google.com/github/BankNatchapol/Comparison-Of-Quantum-Gradient/blob/main/concept_implementation/qiskit%26pennylane_lcu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit

# **Qiskit implementation of differentiation using Linear Combination of Unitaries (LCU)**

In [2]:
#General imports
import numpy as np

#Operator Imports
from qiskit.aqua.operators import Z, X, I, StateFn, CircuitStateFn, SummedOp
from qiskit.aqua.operators.gradients import Gradient, NaturalGradient, QFI, Hessian

#Circuit imports
from qiskit.circuit import QuantumCircuit, QuantumRegister, Parameter, ParameterVector, ParameterExpression
from qiskit.circuit.library import EfficientSU2

/usr/local/lib/python3.7/dist-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')


In [3]:
# Instantiate the quantum state
a = Parameter('a')
b = Parameter('b')
q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.h(q)
qc.rz(a, q[0])
qc.rx(b, q[0])

# Define the Hamiltonian with fixed coefficients
H = 0.5 * X - 1 * Z
# Define the parameters w.r.t. we want to compute the gradients
params = [a, b]
# Define the values to be assigned to the parameters
value_dict = { a: np.pi / 4, b: np.pi} # initial parameter values

# Combine the Hamiltonian observable and the state into an expectation value operator
op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)
print(op)

ComposedOp([
  OperatorMeasurement(SummedOp([
    0.5 * X,
    -1.0 * Z
  ])),
  CircuitStateFn(
        ┌───┐┌───────┐┌───────┐
  q0_0: ┤ H ├┤ Rz(a) ├┤ Rx(b) ├
        └───┘└───────┘└───────┘
  )
])


/usr/local/lib/python3.7/dist-packages/qiskit/aqua/operators/operator_base.py:46: DeprecationWarning: The package qiskit.aqua.operators is deprecated. It was moved/refactored to qiskit.opflow (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua.operators', 'qiskit.opflow', 'qiskit-terra')


In [4]:
state_grad = Gradient(grad_method='lin_comb').convert(operator=op, params=params)

# Print the operator corresponding to the gradient
print(state_grad)

ListOp([
  SummedOp([
    0.5 * ComposedOp([
      OperatorMeasurement(ZZ) * 2.0,
      CircuitStateFn(
                                  ┌───┐          ┌───────┐┌───────┐┌───┐
                            q0_0: ┤ H ├────────■─┤ Rz(a) ├┤ Rx(b) ├┤ H ├
                                  ├───┤┌─────┐ │ └─┬───┬─┘└───────┘└───┘
      work_qubit_lin_comb_grad_0: ┤ H ├┤ Sdg ├─■───┤ H ├────────────────
                                  └───┘└─────┘     └───┘                
      ) * 0.7071067811865476
    ]),
    -1.0 * ComposedOp([
      OperatorMeasurement(ZZ) * 2.0,
      CircuitStateFn(
                                  ┌───┐          ┌───────┐┌───────┐
                            q0_0: ┤ H ├────────■─┤ Rz(a) ├┤ Rx(b) ├
                                  ├───┤┌─────┐ │ └─┬───┬─┘└───────┘
      work_qubit_lin_comb_grad_0: ┤ H ├┤ Sdg ├─■───┤ H ├───────────
                                  └───┘└─────┘     └───┘           
      ) * 0.7071067811865476
    ])
  ]),
  SummedOp([
    0.5 * Compos

In [5]:
# Assign the parameters and evaluate the gradient
value_dict = { a: np.pi / 4, b: np.pi}
state_grad_result = state_grad.assign_parameters(value_dict).eval()
state_grad_result

[(-0.35355339059327345-2.48e-16j), (0.7071067811865474+1.64e-16j)]

In [ ]:
!pip install pennylane-sf

# **Re-implementing of LCU using Pennylane**

In [7]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import GradientDescentOptimizer

import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

In [8]:
num_wires = 2
dev = qml.device("default.qubit", wires=num_wires)

In [32]:
param_gates = ['RX', 'RY', 'RZ']

def projector1(wires):
  projector = np.zeros((2, 2))
  projector[1,1] = 1
  qml.QubitUnitary(projector, wires=wires)

def str2gate(gate):
  if gate == 'H':
    return qml.Hadamard
  elif gate == 'X':
    return qml.PauliX
  elif gate == 'Y':
    return qml.PauliY
  elif gate == 'Z':
    return qml.PauliZ
  elif gate == 'RX':
    return qml.RX
  elif gate == 'RY':
    return qml.RY
  elif gate == 'RZ':
    return qml.RZ
  elif gate == 'Projector1':
    return projector1
  else:
    assert 0, "Gate not support."

def print_ansatz(ansatz, coeffs, wires):
  @qml.qnode(dev)
  def ansatz_construct(ansatz, coeffs, wires):
    for i, coeff in enumerate(coeffs):
      if not coeff:
        str2gate(ansatz[i].numpy())(wires=wires[i].numpy())
      else:
        str2gate(ansatz[i].numpy())(coeff.numpy(), wires=wires[i].numpy())

    return qml.state()
  print(qml.draw(ansatz_construct)(ansatz, coeffs, wires))

Creating ansatz

In [33]:
ansatz = ['H', 'RZ', 'RX']
params_init = np.array([None, np.pi/4, np.pi])
wires = [0, 0, 0]

print_ansatz(ansatz, params_init, wires)

 0: ──H──RZ(0.785)──RX(3.14)──┤ State 



Hamiltonian $H = 0.5X - Z$

In [34]:
H = ['X', 'Z']
ham_wires = [0, 0]
ham_coeffs = [0.5, -1.0]

In [35]:
def controlStr(gate):
  if gate == 'RX':
    return qml.CNOT
  elif gate == 'RY':
    return qml.CY
  elif gate == 'RZ':
    return qml.CZ
  else:
    assert 0, "Gate not support."


def lcu_term(ansatz, params, wires, diff_index):
  qml.Hadamard(wires=1)
  qml.adjoint(qml.S)(wires=1)

  for i, gate in enumerate(ansatz):
    if gate in param_gates:
      if i == diff_index:
        controlStr(gate)(wires=[1, 0])
      str2gate(gate)(params[i], wires=wires[i])
    else:
      str2gate(gate)(wires=wires[i])
    
  qml.Hadamard(wires=1)


def lcu_term_adjointed(ansatz, params, wires, diff_index):
  ansatz = [a for a in reversed(ansatz)]
  params = params[::-1]

  qml.Hadamard(wires=1)
  for i, gate in enumerate(ansatz):
    if gate in param_gates:
      str2gate(gate)(-params[i], wires=wires[i])
      if i == len(ansatz)-diff_index-1:
        controlStr(gate)(wires=[1, 0])
    else:
      str2gate(gate)(wires=wires[i])

  qml.S(wires=1)
  qml.Hadamard(wires=1)


@qml.qnode(dev)
def lcu(ansatz, H, ham_wires, params, wires, diff_index):
  ansatz = [a.numpy() for a in ansatz]
  wires = [w.numpy() for w in wires]

  lcu_term(ansatz, params, wires, diff_index)
  str2gate(H)(wires=ham_wires)
  str2gate('Z')(wires=1)
  lcu_term_adjointed(ansatz, params, wires, diff_index)

  return qml.state()

def lcu_gradients(ansatz, H, ham_wires, ham_coeffs, params, wires):
  gradients = []
  for i in range(len(ansatz)):
    if ansatz[i] in param_gates:
      gradients.append(sum([lcu(ansatz, H[h], ham_wires[h], params, wires, i)[0].numpy()*ham_coeffs[h] for h in range(len(H))]))

  return gradients

In [36]:
lcu_gradients(ansatz, H, ham_wires, ham_coeffs, params_init, wires)

[(-0.35355339059327373+0j), (0.7071067811865471+7.850462293418875e-17j)]